# Дмитрий Ильин. ДЗ №3 - Задача QA.

Базовая модель bert-base-multilingual-cased.\
F1 = 0.7583

In [ ]:
import os
import time
import json
import pandas as pd
import numpy as np

In [ ]:
import torch

In [ ]:
#!pip install simpletransformers datasets

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
from datasets import load_dataset

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
root_path = "/content/drive/My Drive/MIPT/NLP/HW3/"
output_path = "/content/drive/My Drive/MIPT/NLP/HW3/output/"
logs_path = "/content/drive/My Drive/MIPT/NLP/HW3/logs/"
models_path = "/content/drive/My Drive/MIPT/NLP/HW3/models/"

# 1. Загрузка и подготовка данных

Ниже происходит загрузка датасета с последующим преобразованием в формат simpletransformers - https://simpletransformers.ai/docs/qa-data-formats/

In [ ]:
dataset = load_dataset("sberquad")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

В процессе работы с датасетом SberSquad обнаружилось, что в ответах на некоторые вопросы answer_start указывает на неверную позицию в context, поэтому была добавлена функция проверки данных.

In [ ]:
def check_data(data):
    missing_answers = 0
    total_answers = 0
    example_bad_answer = None

    for sample in data:
        context = sample['context']
        for answer_text, answer_start in zip(sample['answers']['text'], sample['answers']['answer_start']):
            total_answers += 1
            end_idx = answer_start + len(answer_text)
            if context[answer_start:end_idx] != answer_text:
                missing_answers += 1
                if not example_bad_answer:
                    example_bad_answer = {
                        'ID': sample['id'],
                        'context': context,
                        'start_idx': answer_start,
                        'expected': answer_text,
                        'real': context[answer_start:end_idx]
                    }

    print(f"Число ответов с неверным answer_start: {missing_answers}/{total_answers}")
    if example_bad_answer:
        print("Пример:")
        print(f"ID: {example_bad_answer['ID']}")
        print(f"Контекст: {example_bad_answer['context']}")
        print(f"Позиция: {example_bad_answer['start_idx']}")
        print(f"Ожидается: {example_bad_answer['expected']}")
        print(f"Реально: {example_bad_answer['real']}")


In [ ]:
check_data(dataset["train"])

Число ответов с неверным answer_start: 10237/45328
Пример:
ID: 28101
Контекст: В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.
Позиция: 438
Ожидается: нитевидные водоросли, грибные нити
Реально: ны нитевидные водоросли, грибные н


In [ ]:
check_data(dataset["validation"])

Число ответов с неверным answer_start: 1108/5036
Пример:
ID: 29930
Контекст: Сверхкороткие импульсы лазерного излучения используются в лазерной химии для запуска и анализа химических реакций. Здесь лазерное излучение позволяет обеспечить точную локализацию, дозированность, абсолютную стерильность и высокую скорость ввода энергии в систему. В настоящее время разрабатываются различные системы лазерного охлаждения, рассматриваются возможности осуществления с помощью лазеров управляемого термоядерного синтеза. Лазеры используются и в военных целях, например, в качестве средств наведения и прицеливания. Рассматриваются варианты создания на основе мощных лазеров боевых систем защиты воздушного, морского и наземного базирования.
Позиция: 113
Ожидается: Лазерное
Реально: . Здесь 


In [ ]:
check_data(dataset["test"])

Число ответов с неверным answer_start: 0/23936


А теперь приведем данные из датасета к формату simpletransformers, а также попробуем исправить неверные значения answer_start, а если в итоге значение не находится, то ответ удаляется из датасета. Если на вопрос в итоге не будет ни одного ответа, то и вопрос также удаляется из датасета.

In [ ]:
def format_for_simpletransformers(data):
    formatted_data = []

    correct_starts_found = 0
    answers_removed = 0
    questions_removed = 0
    initial_question_count = len(data)

    for sample in data:
        context = sample['context']
        formatted_sample = {
            'context': context,
            'is_impossible': False,
            'qas': [{
                'id': sample['id'],
                'question': sample['question'],
                'answers': []
            }]
        }

        for answer_text, answer_start in zip(sample['answers']['text'], sample['answers']['answer_start']):
            # Проверяем, совпадает ли текст ответа с answer_start
            if context[answer_start:answer_start + len(answer_text)] != answer_text:
                # Если не совпадает, пытаемся найти правильный индекс
                correct_start = context.find(answer_text)

                # Если новый индекс найден, обновляем answer_start
                if correct_start != -1:
                    answer_start = correct_start
                    correct_starts_found += 1
                else:
                    answers_removed += 1
                    continue

            formatted_sample['qas'][0]['answers'].append({
                'text': answer_text,
                'answer_start': answer_start
            })

        if formatted_sample['qas'][0]['answers']:
            formatted_data.append(formatted_sample)
        else:
            questions_removed += 1

    print(f"Всего вопросов: {initial_question_count}")
    print(f"Исправлено ответов: {correct_starts_found}")
    print(f"Удалено ответов: {answers_removed}")
    print(f"Удалено вопросов: {questions_removed}")
    print("")

    return formatted_data

In [ ]:
train_data = format_for_simpletransformers(dataset['train'])
validation_data = format_for_simpletransformers(dataset['validation'])
test_data = format_for_simpletransformers(dataset['test'])

Всего вопросов: 45328
Исправлено ответов: 2187
Удалено ответов: 8050
Удалено вопросов: 8050

Всего вопросов: 5036
Исправлено ответов: 242
Удалено ответов: 866
Удалено вопросов: 866

Всего вопросов: 23936
Исправлено ответов: 0
Удалено ответов: 0
Удалено вопросов: 0



In [ ]:
len(train_data)

37278

Теперь проведем проверку скрорректированных и отформатированных данных.

In [ ]:
def check_formatted_data(data):
    missing_answers = 0
    total_answers = 0
    for item in data:
        context = item['context']
        for qa in item['qas']:
            for answer in qa['answers']:
                total_answers += 1
                start_idx = answer['answer_start']
                end_idx = start_idx + len(answer['text'])
                if context[start_idx:end_idx] != answer['text']:
                    missing_answers += 1

    print(f"Missing answers: {missing_answers}/{total_answers}")

check_formatted_data(train_data)
check_formatted_data(validation_data)
check_formatted_data(test_data)

Missing answers: 0/37278
Missing answers: 0/4170
Missing answers: 0/23936


Часть неверных значений answer_start удалось поправить.

# 3. Подбор гиперпараметров и обучение



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(f"GPU: {torch.cuda.get_device_name()}")
else:
    print("CPU")

GPU: Tesla V100-SXM2-16GB


In [ ]:
def get_default_model_args():
    model_args = QuestionAnsweringArgs()
    model_args.evaluate_during_training = True
    model_args.evaluate_during_training_verbose = True
    model_args.num_train_epochs = 1
    model_args.learning_rate = 3e-5
    model_args.train_batch_size = 8
    model_args.eval_batch_size = 8
    model_args.overwrite_output_dir = True
    model_args.reprocess_input_data = True
    model_args.output_dir = output_path
    model_args.best_model_dir = models_path
    model_args.tensorboard_dir = logs_path
    model_args.manual_seed = 42
    model_args.weight_decay = 0.01
    model_args.save_eval_checkpoints = False
    model_args.save_model_every_epoch = False
    model_args.save_best_model = True
    model_args.scheduler = "linear_schedule_with_warmup"

    return model_args

In [ ]:
def get_model(model_args = None):
    if model_args is None:
        model_args = get_default_model_args()
    return QuestionAnsweringModel(
        "bert",
        "bert-base-multilingual-cased",
        args=model_args
    )

In [ ]:
def compute_f1(predictions, references):
    if len(predictions) != len(references):
        return 0

    pred_binary = [1 if p == r else 0 for p, r in zip(predictions, references)]
    ref_binary = [1 if r else 0 for r in references]

    result = sklearn.metrics.f1_score(ref_binary, pred_binary)

    return result

## 3.1. Подбор гиперпараметров
На данном этапе перебираются значения следующих гиперпараметров: learning_rate и num_train_epoch. На перебор бОльшего числа гиперпараметров, к сожалению, не хватило времени и ресурсов.\
По результату мы получаем словарь с лучшими гиперпараметрами на основе метрики F1, который сохраняется на диск.

In [ ]:
learning_rates = [5e-5, 3e-5, 1e-5]
num_train_epochs = [2, 3, 4]

best_f1 = 0
best_params = {}

for lr in learning_rates:
    for epoch in num_train_epochs:
        print(f"Обучаем на: learning_rate={lr}, num_train_epochs={epoch}")

        current_params = {
            "learning_rate": lr,
            "num_train_epochs": epoch
        }

        model_args = get_default_model_args()
        model_args.best_model_dir = None
        for key, value in current_params.items():
            setattr(model_args, key, value)

        model = get_model(model_args)
        _, result = model.train_model(train_data, eval_data=validation_data, f1=compute_f1)

        f1 = result['f1'][-1]
        print(f"F1: {f1}")
        if f1 > best_f1:
            best_f1 = f1
            best_params = current_params

print(f"Лучшее значение F1: {best_f1} с параметрами: {best_params}")

In [ ]:
with open(os.path.join(root_path, "best_params.json"), "w") as file:
    json.dump(best_params, file)

## 3.2. Обучение

Да данном этапе мы обучаем нашу модель на подобранных гиперпараметрах и получаем итоговую метрику F1.

In [ ]:
with open(os.path.join(root_path, "best_params.json"), "r") as file:
    best_params = json.load(file)

print(best_params)

{'learning_rate': 3e-05, 'num_train_epochs': 3}


In [ ]:
model_args = get_default_model_args()
model_args.best_model_dir = os.path.join(models_path, f"final")
model_args.output_dir = os.path.join(output_path, f"final")
model_args.tensorboard_dir = os.path.join(logs_path, f"final")
for key, value in best_params.items():
    setattr(model_args, key, value)

model = get_model(model_args)
_, result = model.train_model(train_data, eval_data=validation_data, f1=compute_f1)
f1 = result['f1'][-1]
print(f"F1: {f1}")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
convert squad examples to features:   0%|          | 0/37278 [00:00<?, ?it/s]Could not find answer: '' vs. 'Национальное Рейтинговое Агентство'
Could not find answer: '' vs. 'Бхагавадгита'
Could not find answer: '' vs. 'Everybody'
Could not find answer: '' vs. 'Георгики'
convert squad examples to features:  25%|██▍       | 9319/37278 [00:31<00:57, 484.60it/s]Could not find answer: '' vs. 'Насьональ'
Could not find answer: '' vs. 'История Рима'
Could not find answer: '' vs. 'Машина времени'
Could not find answer: '' vs. 'Машина времени'
add example index and unique id: 100%|██████████| 37278/37278 [00:00<00:00, 770978.76it/s]


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/4807 [00:00<?, ?it/s]



convert squad examples to features:   0%|          | 0/4170 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/4170 [00:04<5:03:22,  4.37s/it]

convert squad examples to features:  12%|█▏        | 501/4170 [00:04<00:23, 156.90it/s]

convert squad examples to features:  36%|███▌      | 1501/4170 [00:04<00:04, 569.29it/s]

convert squad examples to features:  72%|███████▏  | 3001/4170 [00:06<00:01, 628.74it/s]

convert squad examples to features: 100%|██████████| 4170/4170 [00:07<00:00, 570.25it/s]


add example index and unique id: 100%|██████████| 4170/4170 [00:00<00:00, 751202.49it/s]


Running Evaluation:   0%|          | 0/538 [00:00<?, ?it/s]

...



convert squad examples to features:   0%|          | 0/4170 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/4170 [00:03<4:31:22,  3.91s/it]

convert squad examples to features:  12%|█▏        | 501/4170 [00:04<00:27, 135.11it/s]

convert squad examples to features:  36%|███▌      | 1501/4170 [00:05<00:05, 468.66it/s]

convert squad examples to features:  48%|████▊     | 2001/4170 [00:05<00:03, 632.14it/s]

convert squad examples to features:  72%|███████▏  | 3001/4170 [00:07<00:01, 618.26it/s]

convert squad examples to features: 100%|██████████| 4170/4170 [00:07<00:00, 565.33it/s]


add example index and unique id: 100%|██████████| 4170/4170 [00:00<00:00, 499507.29it/s]


Running Evaluation:   0%|          | 0/538 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 4170/4170 [00:07<00:00, 584.66it/s]

add example index and unique id: 100%|██████████| 4170/4170 [00:00<00:00, 670612.62it/s]


Running Evaluation:   0%|          | 0/538 [00:00<?, ?it/s]

F1: 0.7583081570996979


# 4. Предсказание
Несмотря на то что в тестовом датасете отсутствуют метки, позволяющие оценить правильность ответов, я все-таки решил провести оценку обученной модели "на глаз".

In [ ]:
results, pred_list = model.eval_model(test_data)

add example index and unique id: 100%|██████████| 23936/23936 [00:00<00:00, 752669.79it/s]


Running Evaluation:   0%|          | 0/3099 [00:00<?, ?it/s]

In [ ]:
predictions_dict = pred_list['similar_text']
qa_list = [{'question': item.get('question', 'N/A'), 'answer': item.get('predicted', 'N/A')} for item in predictions_dict.values() if 'question' in item and 'predicted' in item]
qa_df = pd.DataFrame(qa_list)
qa_df.sample(25)

,question,answer
7887,Контроль над какой провинцией намеревалась пол...,Сисплатина
8216,ближайшим потомком какой династии был Филипп V...,Капетингов
13947,Как называется устойчивая совокупность идейных...,Моральный дух личного состава
8141,В каком году РН Протон дважды падали в Караган...,В 1999 году
13396,С какими актерами повстречался Мольер в самом ...,с комедиантами Жозефом и Мадленой Бежар
12857,В каком веке в Испании началась католическая р...,в XV веке
5279,Какой материк служит интересным примером сезон...,Евразия
9960,На кого Кутейба обрушил наиболее жестокие репр...,на учёных Хорезма
10576,С каким укорочением конечностей справляются в ...,до 50 см и более
9124,С кем у лужицких сербов крепкие исторические с...,с Польшей и Чехией


# 5. Выводы
Как и в предыдущей домашней работе по NER, комбинация модели bert-base-multilingual-cased и библиотеки simpletransormers показала неплохой результат c метрикой F1 = 0.7583. Также ответы на тестовые вопросы кажутся в основной массе точными.\
Подход с подбором гиперпараметров оправдал себя и позволил получить больший скор, чем дефолтные параметры. Очень жаль, что у меня не было ресурсов и времени на перебор большего количества гиперпарамтеров и обучения нескольких моделей в ансанмбле - уверен, что это помогло бы поднять скор еще выше.